## OSEMN Approach
1 - Obtaining the data 2 - Scrubbing the data 3 - Exploring the data 4 - Modeling the data 5 - iNterpreting the results


![title](picture2.png)


## Questions:

1) What are the main factors to predict the price of a house?

2) How much does a middle income family should invest to buy a standard house?
   
   or What are the properties of a house you can buy in King County if your budget is $400,000???
    
3) How can a middle income family increase the price of their house by %20 by spending %5 of the house cost???

4) Basement
 

In [1]:
%autosave 10

Autosaving every 10 seconds


In [2]:
# # Print all the interactive output without resorting to print
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

## Import the necessary packages

First, we’ll import all of our necessary packages.

In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
# import sklearn as sk 
from statsmodels.formula.api import ols


In [4]:
# ## Display more information 
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

## Import the data set

Next, we’ll import our data and see what we’re working with. 

In [6]:
df=pd.read_csv('notebooks/kc_house_data.csv')

FileNotFoundError: [Errno 2] File b'notebooks/kc_house_data.csv' does not exist: b'notebooks/kc_house_data.csv'

Lets see what the data looks like by functions head(), tail(), shape(), info() and describe().

In [ ]:
df.head()

### Drop unnecaessary columns

In [ ]:
df=df.drop(['id','date'],axis=1)

# Scraping and cleaning data

In [ ]:
df.shape

In [ ]:
df.info()

### Numerical Data Stored as Strings

In [ ]:
df.sqft_basement.value_counts()

In [ ]:
df['sqft_basement']=df['sqft_basement'].map(lambda x:float(x.replace('?', '0')))


In [ ]:
#Replace 0 with difference between sqft_above and sqft_living
df['sqft_basement'].replace(to_replace='0', value = abs(df['sqft_living']-df['sqft_above']), inplace=True)


In [ ]:
df['bedrooms'].replace(to_replace='33', value = 3, inplace=True)


In [ ]:
df.describe().round(1)

### Buraya describe ile ilgili yorum yaz
Most houses were build around 1975; sold for $450K; had square footage of living spaces around 1910. You can also note, that distributions of price and variables such as sqft_lot are skewed to the upper tail.



## Detecting and Dealing With Null Values

In [ ]:
print(df.isna().sum())

### Buraya na values  ile ilgili yorum yaz


# waterfront ve view 

In [ ]:
df['view'].value_counts()

In [ ]:
df['view'].isna().sum()

In [ ]:
feats = ['view', 'waterfront']
df[feats].corr()

In [ ]:
print('Number missing both:',
      len(df[(df.view.isnull())
      & (df.waterfront.isnull())]))

In [ ]:
len(df[(df.view==2)&(df.waterfront.isna())])

In [ ]:
len(df[(df.view==3)&(df.waterfront>0)])

In [ ]:
df['view'].fillna(0,inplace=True)

In [ ]:
df['waterfront'].value_counts()

In [ ]:
df['waterfront'].isna().sum()

In [ ]:
df['waterfront'].fillna(0,inplace=True)

In [ ]:
feats = ['view', 'waterfront']
df[feats].corr()

In [ ]:
df.yr_renovated.fillna(0, inplace=True)

### Outliers

In [ ]:
Q1 = df.quantile(0.005)
Q3 = df.quantile(0.98)
IQR = Q3 - Q1

df_no_outliers = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

## Normalizing the data

In [ ]:
def norm_feat(series):
    return (series - series.mean())/series.std()
for feat in ['bathrooms', 'sqft_living', 
       'sqft_lot','sqft_above', 'sqft_basement', 'yr_built','yr_renovated','sqft_living15', 'sqft_lot15']:
    df[feat] = norm_feat(df[feat])


In [ ]:
df.describe().round(2)

In [ ]:
df.to_csv("king_county_dataset_cleaned.csv", index=False)


## One-Hot Encoding Categorical Columns


#### Identifying Categorical Variables

These columns can be thought as categorical values. Bedrooms, floors, waterfront, wiev, grade, zipcode and year renovated.
Let's explore them one by one.

In [ ]:
df['bedrooms'] = df['bedrooms'].astype('category')
df['floors'] = df['floors'].astype('category')
df['waterfront'] = df['waterfront'].astype('category')
df['view'] = df['view'].astype('category')
df['condition'] = df['condition'].astype('category')
df['grade'] = df['grade'].astype('category')


In [ ]:
feats = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 
       'sqft_living15', 'sqft_lot15']
df_feats = df[feats]
df_feats = pd.get_dummies(df_feats, drop_first=True)

In [ ]:
df_feats.info()

In [ ]:
df_feats.head()

'zipcode', 'lat', 'long',

# Exploring the data

In [ ]:
df.head()

In [ ]:
df_feats.head()

In [ ]:
sns.pairplot(df)

## Check for Linearity

jointplots

## Check for Multicollinearity

In [ ]:
# df_target = df_no_outliers.loc['price']
df_heatmap=df.drop(['price','zipcode', 'lat', 'long'],axis=1)
plt.figure(figsize=(20,20))
ax=sns.heatmap(df_heatmap.corr(), center=0, linewidths=.5,annot=True);
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

In [ ]:
df_heatmap.corr() > 0.75


In [ ]:
df_feats.info()

In [ ]:
# # df=df.drop(['sqft_living'], axis=1)
# df_feats=df_feats.drop(['sqft_living'], axis=1)
# df_feats=df_feats.drop(['sqft_living'], axis=1)

In [ ]:
df_feats.columns

In [ ]:
outcome = 'price'
x_cols = ['bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15',
       'sqft_lot15', 'bedrooms_2', 'bedrooms_3', 'bedrooms_4', 'bedrooms_5',
       'bedrooms_6', 'bedrooms_7', 'bedrooms_8', 'bedrooms_9', 'bedrooms_10',
       'bedrooms_11', 'bedrooms_33', 'floors_1_5', 'floors_2_0', 'floors_2_5',
       'floors_3_0', 'floors_3_5', 'waterfront_1_0', 'view_1_0', 'view_2_0',
       'view_3_0', 'view_4_0', 'condition_2', 'condition_3', 'condition_4',
       'condition_5', 'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8',
       'grade_9', 'grade_10', 'grade_11', 'grade_12', 'grade_13']

In [ ]:
# df_feats.columns = [col.replace('.', '_', inplace=True) for col in df_feats.columns]
# df_feats.columns = [col.replace(' ', '_', inplace=True) for col in df_feats.columns]
df_feats.columns = df_feats.columns.str.replace(r"[.]", "_")

In [ ]:
df_feats.columns

In [ ]:
# Fitting the actual model
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=df_feats).fit()
model.summary()

## notes


basement to new feature



subsampling

regularisation 



future work:  garaj, pool, okul gradeleri

## web sites
hexbins with longitude and latitude 
https://medium.com/@mattheweparker/visualizing-data-with-hexbins-in-python-39823f89525e  

categorigal data 
https://medium.com/@rabinpoudyal1995/encoding-categorical-data-in-python-fab150d6e21b

All houses are sold within 390 days, therefore I will not need to adjust the price.
https://github.com/Burton-David/KingsCountyHousingLR/blob/master/student.ipynb




https://www.incomebyzipcode.com/washington/98109

https://medium.com/@allisonkelly42/exploratory-data-analysis-with-the-king-county-housing-data-c6d3f7eadc57